# Format data for housing project
1. [Format FIPs, county name and date](#1.-Format-FIPs,-county-name-and-date)
2. [Select categories (median_listing_price, median_days_on_market, new_listing_count, active_listing_count)](#2.-Select-categories)
3. [Filter out counties that don't have data for every month](#3.-Filter-out-counties-that-don't-have-data-for-every-month)
4. [Filter out counties with less than a certain amount of active listings](#4.-Filter-out-counties-with-less-than-a-certain-amount-of-active-listings)
5. Format data for the latest month
6. [Format metadata](#6.-Format-metadata)
7. [Format timeseries data and save to file for each fips code](7.-Format-timeseries-data-and-save-to-file-for-each-fips-code)
8. Format national data

In [2]:
import pandas as pd
import json
import warnings

warnings.filterwarnings(action='ignore')

In [3]:
df = pd.read_csv('data/RDC_Inventory_Core_Metrics_County_History.csv')

## Common functions

In [21]:
def format_date(x):
    return str(x)[0:4] + '-' + str(x)[4:6] + '-01'

def format_fips(x):
    return str(x).zfill(5)

selected_categories = ['month_date', 'quality_flag', 'median_listing_price', 'median_days_on_market', 'new_listing_count', 'active_listing_count', 'households', 'population'];


## 1. Format FIPs, county name and date

#### Format date

In [7]:
df['month_date'] = df['month_date_yyyymm'].apply(format_date)

#### Format FIPs

In [6]:
df['county_fips'] = df['county_fips'].apply(format_fips)

In [16]:
df_meta = pd.read_csv('data/county_census_data.csv')
df_meta.columns = ['county_fips', 'county_name', 'households', 'population']
df_meta['county_fips'] = df_meta['county_fips'].apply(lambda x: str(x).zfill(5))
df_meta = df_meta.set_index('county_fips')

#### Merge formatted datasets

In [18]:
df_merge = pd.merge(df, df_meta, on='county_fips', suffixes=('_old', ''))

## 2. Select categories
- median_listing_price
- median_days_on_market
- new_listing_count
- active_listing_count)

In [22]:
df_selected = df_merge[selected_categories + ['county_name', 'county_fips']]


## 3. Filter out counties that don't have data for every month

In [24]:
df_six_years = df_selected[df_selected['month_date'] >= '2018-01-01']

In [25]:
df_totals = pd.DataFrame(df_six_years.groupby('county_fips')['month_date'].nunique().sort_values(ascending=False))

In [26]:
unique_month_count = df_totals['month_date'].max()
unique_month_count

np.int64(88)

In [27]:
df_complete = df_totals[df_totals['month_date'] == unique_month_count]

In [28]:
df_complete

,month_date
county_fips,
01001,88
39073,88
39055,88
39057,88
39059,88
...,...
21089,88
21021,88
21093,88


In [29]:
df_selected_counties = df_six_years[df_six_years['county_fips'].isin(df_complete.index.tolist())]

## 4. Filter out counties with less than a certain amount of active listings

In [30]:
df_non_zero_listings = df_selected_counties[df_selected_counties['active_listing_count'] > 10]
df_non_zero_listings = df_non_zero_listings[~df_selected_counties['active_listing_count'].isna()]

In [31]:
df_listing_totals = pd.DataFrame(df_non_zero_listings.groupby('county_fips')['month_date'].nunique())

In [32]:
df_filtered_counties = df_six_years[df_six_years['county_fips'].isin(df_listing_totals.index)]

## 5. Format data for the latest month

In [33]:
latest_month = df_filtered_counties['month_date'].max()
last_year = str(int(latest_month[0:4]) - 1) + latest_month[4:]

In [34]:
latest_month, last_year

('2025-04-01', '2024-04-01')

In [35]:
df_latest_month = df_filtered_counties[(df_filtered_counties['month_date'] == latest_month) | (df_filtered_counties['month_date'] == last_year)]


In [37]:
df_pivot_yoy = df_latest_month.pivot(index='county_fips', columns='month_date', values=['median_listing_price', 'active_listing_count'])

df_pivot_yoy['median_listing_price_yoy'] = (df_pivot_yoy[('median_listing_price', latest_month)] - df_pivot_yoy[('median_listing_price', last_year)]) / df_pivot_yoy[('median_listing_price', last_year)]
df_pivot_yoy['active_listing_count_yoy'] = (df_pivot_yoy[('active_listing_count', latest_month)] - df_pivot_yoy[('active_listing_count', last_year)]) / df_pivot_yoy[('active_listing_count', last_year)]
df_pivot_yoy = df_pivot_yoy.drop([('median_listing_price', last_year), ('active_listing_count', last_year)], axis=1)
df_pivot_yoy = df_pivot_yoy.reset_index()
df_pivot_yoy.columns = df_pivot_yoy.columns.droplevel(-1)

df_pivot_yoy = df_pivot_yoy.merge(df_meta, how='left', on='county_fips')

In [39]:
latest_month_json = {
    'data': json.loads(df_pivot_yoy.to_json(orient='records')),
    'latest_month': latest_month
}

with open('../public/data/latest.json', 'w') as outfile:
    json.dump(latest_month_json, outfile)

In [40]:
df_pivot_yoy.to_json('../public/data/latest.json', orient='records')

## 6. Format metadata
Save to `../src/assets/fips.json` as `{'county_name': '', 'county_fips': ''}`

### Format county metadata

In [40]:
df_unique_counties = df_filtered_counties.drop_duplicates('county_fips')

In [41]:
df_unique_counties[['county_name', 'county_fips']].to_json('../src/assets/fips.json', orient='records')

### Format state metadata

In [112]:
df_states = df_population[df_population['FIPS'].str[2:] == '000']
df_states = df_states[['FIPS', 'STNAME']]
df_states['FIPS'] = df_states['FIPS'].str[0:2]
df_states.columns = ['fips', 'name']
df_states.to_json('./../public/data/states.json', orient='records')

## 7. Format timeseries data and save to file for each fips code
Save to `../public/data/counties/[FIPS].json`

In [87]:
def format_moving_average(df):
    df = df.sort_values('month_date')
    df['median_listing_price_rolling'] = df['median_listing_price'].rolling(window=12).mean()
    df['active_listing_count_rolling'] = df['active_listing_count'].rolling(window=12).mean()
    return df

In [88]:
for county_fips in df_unique_counties['county_fips'].tolist():
    df_selected_fips = df_filtered_counties[df_filtered_counties['county_fips'] == county_fips]
    # print(df_selected_fips)
    df_selected_fips['month_date'] = df_selected_fips['month_date'].apply(str)
    df_selected_fips = format_moving_average(df_selected_fips)
    obj = {
        # 'median_listing_price', 'median_days_on_market', 'new_listing_count', 'active_listing_count']
        'county_name': df_selected_fips.reset_index()['county_name'][0],
        'latest': df_pivot_yoy[df_pivot_yoy['county_fips'] == county_fips].to_dict(orient='records')[0],
        'median_listing_price': json.loads(df_selected_fips[['month_date', 'median_listing_price']].to_json(orient='values')),
        'median_listing_price_rolling': json.loads(df_selected_fips[['month_date', 'median_listing_price_rolling']].to_json(orient='values')),
        'active_listing_count_rolling': json.loads(df_selected_fips[['month_date', 'active_listing_count_rolling']].to_json(orient='values')),
        'active_listing_count': json.loads(df_selected_fips[['month_date', 'active_listing_count']].to_json(orient='values')),
        'median_days_on_market': json.loads(df_selected_fips[['month_date', 'median_days_on_market']].to_json(orient='values'))
    }
    with open('../public/data/counties/%s.json' % county_fips, 'w') as outfile:
        json.dump(obj, outfile)

## 7. Format national data

In [91]:
df_national = pd.read_csv('data/RDC_Inventory_Core_Metrics_Country_History.csv')

In [80]:
df_national = df_national[df_national['country'] == 'United States']
df_national['month_date'] = df_national['month_date_yyyymm'].apply(format_date)
df_national = df_national[selected_categories]
df_national = format_moving_average(df_national)

In [81]:
df_national = df_national.sort_values('month_date')

In [82]:
df_national.iloc[len(df_national)-1]

month_date                         2025-04-01
quality_flag                              0.0
median_listing_price                 431250.0
median_days_on_market                    50.0
new_listing_count                    471788.0
active_listing_count                 959251.0
median_listing_price_rolling    423697.833333
active_listing_count_rolling         888888.5
Name: 0, dtype: object

In [83]:
df_national['median_listing_price_yoy'] = df_national['median_listing_price'].pct_change(12)
df_national['active_listing_count_yoy'] = df_national['active_listing_count'].pct_change(12)


In [84]:
obj = {
    # 'median_listing_price', 'median_days_on_market', 'new_listing_count', 'active_listing_count']
    'latest': df_national.iloc[len(df_national)-1].to_dict(),
    'median_listing_price': json.loads(df_national[['month_date', 'median_listing_price']].to_json(orient='values')),
    'median_listing_price_yoy': json.loads(df_national[['month_date', 'median_listing_price_yoy']].to_json(orient='values')),
    'median_listing_price_rolling': json.loads(df_national[['month_date', 'median_listing_price_rolling']].to_json(orient='values')),
    'active_listing_count_rolling': json.loads(df_national[['month_date', 'active_listing_count_rolling']].to_json(orient='values')),
    'active_listing_count': json.loads(df_national[['month_date', 'active_listing_count']].to_json(orient='values')),
    'active_listing_count_yoy': json.loads(df_national[['month_date', 'active_listing_count_yoy']].to_json(orient='values')),
    'median_days_on_market': json.loads(df_national[['month_date', 'median_days_on_market']].to_json(orient='values'))
}
with open('../public/data/national.json', 'w') as outfile:
    json.dump(obj, outfile)